Note: this still isn't working as the tickers in the constituents don't match tickers existing now. Maybe work on getting data using the isin from the ishares website? Maybe manually create symbol change table

In [48]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

data = requests.get("https://www.ishares.com/us/products/239726/#tabsAll").content
soup = BeautifulSoup(data)

holdings = soup.find("div", {"id":"holdings"})
date_list_div = holdings.find_all("div","component-date-list")[1] # second one has all dates
date_list_div.find_all("option")

date_list = [option.attrs["value"] for option in date_list_div.find_all("option")]



In [49]:
constituents = pd.Series()

for date_str in date_list:
    data = requests.get(f"https://www.ishares.com/us/products/239726/ishares-core-sp-500-etf/1467271812596.ajax?tab=all&fileType=json&asOfDate={date_str}").content[3:]
    tickers = json.loads(data)
    tickers = [arr[0] for arr in tickers['aaData']]
    date = datetime.strptime(date_str, "%Y%m%d")
    constituents[date] = tickers

In [51]:
constituents.to_csv("constituents.csv")

In [107]:
import ast
c = pd.read_csv('constituents.csv',header=None, index_col=0, squeeze=True, parse_dates=True, converters={1:ast.literal_eval})

In [122]:
c = c.iloc[::-1] # reverse into cronlogical order

In [143]:
from datetime import datetime, timedelta

In [163]:
import pandas_datareader.data as web

data = {}

for i in range(c.index.get_loc("2014-04-30"), len(c)): #for i in range(0, len(c)):
    start = c.index[i].to_pydatetime()
    if i == len(c) - 1:
        end = datetime.now() 
    else:
        end = c.index[i + 1].to_pydatetime() - timedelta(days=1)
    print(f"{start} - {end}")
    for ticker in c[i]:
        try:
            df = web.DataReader(ticker,'iex', start, end)
            if ticker in data:
                data[ticker] = data[ticker].append(df)
            else:
                data[ticker] = df
        except Exception as e:
            print(f"{ticker}: {e}")


2014-06-30 00:00:00 - 2014-07-30 00:00:00
BRKB: 'BRKB'
WAG: 'WAG'
DD: 'DD'
EMC: 'EMC'
TWC: 'TWC'
COV: 'COV'
PCP: 'PCP'
KRFT: 'KRFT'
BHI: 'BHI'
FRX: 'FRX'
SNDK: 'SNDK'
MHFI: 'MHFI'
LO: 'LO'
TYC: 'TYC'
BRCM: 'BRCM'
STJ: 'STJ'
MJN: 'MJN'
RAI: 'RAI'
GMCR: 'GMCR'
HOT: 'HOT'
BFB: 'BFB'
WFM: 'WFM'
CAM: 'CAM'
BLKFDS: 'BLKFDS'
SIAL: 'SIAL'
LLTC: 'LLTC'
COH: 'COH'
CFN: 'CFN'
CSC: 'CSC'
HSP: 'HSP'
PCL: 'PCL'
SWY: 'SWY'
TSO: 'TSO'
MWV: 'MWV'
HAR: 'HAR'
ARG: 'ARG'
SPLS: 'SPLS'
POM: 'POM'
GAS: 'GAS'
FDO: 'FDO'
JOY: 'JOY'
PETM: 'PETM'
TEG: 'TEG'
HCBK: 'HCBK'
CVC: 'CVC'
TE: 'TE'
2014-07-31 00:00:00 - 2014-08-28 00:00:00
BRKB: 'BRKB'
BLKFDS: 'BLKFDS'
BRCM: 'BRCM'
STJ: 'STJ'


KeyboardInterrupt: 

In [145]:
data

{'GHC':                   High         Low        Open       Close  Volume   Adj Close
 Date                                                                          
 2008-01-31  454.501495  441.274933  443.353485  449.546814   41500  342.000702
 2008-02-01  459.214508  448.350464  448.942596  458.356506   43300  348.702789
 2008-02-04  467.522644  454.326294  467.522644  455.480377   29200  346.514771
 2008-02-05  454.078552  442.392761  454.078552  443.558899   30600  337.445221
 2008-02-06  473.510559  444.114807  446.525665  457.474335   58400  348.031799
 2008-02-07  465.148041  453.141998  457.250763  458.090637   41500  348.500549
 2008-02-08  456.797577  440.604218  456.797577  442.296082   50800  336.484680
 2008-02-11  446.652557  437.583069  442.900299  446.652557   18200  339.798950
 2008-02-12  451.927490  445.552856  446.622345  450.755280   16700  342.920044
 2008-02-13  456.175232  444.157104  453.474335  447.214508   25800  340.226379
 2008-02-14  451.305145  446.1148